In [59]:
import os
import pandas as pd
from tqdm import tqdm
import torch
import pyarrow.parquet as pq
import dask.dataframe as dd
import spacy
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, BertTokenizerFast, BertForSequenceClassification, BertConfig
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter
from torch.utils.data import DataLoader, Dataset as TorchDataset
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
import hdbscan
from scipy.spatial.distance import pdist, squareform
import logging
import re
from joblib import Parallel, delayed


class ReviewsKeywords:
    def __init__(self, csv_path, model_path, spacy_model="ru_core_news_lg"):
        self.csv_path = csv_path
        self.model_path = model_path

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if self.device == "cuda":
            import cudf.pandas  # Импортирование cuDF и активация его использования
            cudf.pandas.install()
        os.environ["TOKENIZERS_PARALLELISM"] = "true"  # Включаем параллелизм токенизатора для ускорения
        self.tokenizer_my = BertTokenizerFast.from_pretrained(self.model_path)
         # Загрузка модели для классификации
        self.classification_model = BertForSequenceClassification.from_pretrained(self.model_path).to(self.device)
        # Загрузка базовой модели для получения эмбеддингов
        self.embedding_model = AutoModel.from_pretrained(self.model_path).to(self.device)
        
        # Загрузка модели и токенайзера от Сбербанка
        self.tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/sbert_large_nlu_ru')
        self.embedding_model = AutoModel.from_pretrained('sberbank-ai/sbert_large_nlu_ru').to(self.device)
        
        spacy.prefer_gpu()
        self.nlp = spacy.load(spacy_model, disable=["ner", "tagger", "attribute_ruler", "lemmatizer"])
        
        self.df = pd.read_csv(self.csv_path, nrows=1000)

    @staticmethod
    def clean_text(text):
        text = re.sub(r'[\n\r\t]+|\s{2,}', ' ', text)
        text = re.sub(r'(?<!\.)\s*\.\s*|\s*\.\s*(?!\.)', '. ', text)
        return text.strip().rstrip('.')

    def split_reviews_into_sentences(self, batch):
        cleaned_texts = [self.clean_text(text) for text in batch['corrected_text']]
        docs = list(self.nlp.pipe(cleaned_texts, batch_size=64))
        batch['sentences'] = [[sent.text for sent in doc.sents] for doc in docs]
        return batch

    def process_reviews(self):
        dataset = Dataset.from_pandas(self.df)
        dataset = dataset.map(self.split_reviews_into_sentences, batched=True, batch_size=32)
        self.df = dataset.to_pandas()
        df_exploded = self.df.explode('sentences').reset_index(drop=True)
        df_exploded = df_exploded.drop(columns=[col for col in df_exploded.columns if col.startswith('__index_level_')])
        return Dataset.from_pandas(df_exploded)

    def compute_sentence_embeddings(self, sentences):
        sentences = [str(sentence) for sentence in sentences if isinstance(sentence, str)]
        if not sentences:
            raise ValueError("Input contains no valid strings.")
        inputs = self.tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.embedding_model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

    def compute_embeddings_after_explode(self, batch):
        sentences = batch['sentences']
        valid_sentences = [str(sentence) for sentence in sentences if isinstance(sentence, str)]
        if not valid_sentences:
            batch['sentence_embeddings'] = [[]] * len(sentences)
            return batch
        embeddings = self.compute_sentence_embeddings(valid_sentences)
        embeddings = embeddings.astype(np.float32)
        final_embeddings = []
        embed_idx = 0
        for sentence in sentences:
            if isinstance(sentence, str):
                final_embeddings.append(embeddings[embed_idx])
                embed_idx += 1
            else:
                final_embeddings.append(np.zeros(embeddings.shape[1], dtype=np.float32))
        batch['sentence_embeddings'] = final_embeddings
        return batch

    def apply_embeddings(self, dataset_exploded):
        return dataset_exploded.map(self.compute_embeddings_after_explode, batched=True, batch_size=128)

    def extract_key_thought(self, cluster_sentences):
        sentences = cluster_sentences.split(" | ")
        embeddings = self.compute_sentence_embeddings(sentences)
        centroid = np.mean(embeddings, axis=0)
        similarities = cosine_similarity(embeddings, [centroid])
        key_sentence_index = np.argmax(similarities)
        return sentences[key_sentence_index]

    def count_words(self, cluster_sentences):
        words = cluster_sentences.split()
        return len(words)

    def recluster_large_cluster(self, cluster_sentences, eps=0.1, min_samples=2):
        sentences = cluster_sentences.split(" | ")
        embeddings = self.compute_sentence_embeddings(sentences)
        re_clustering = DBSCAN(eps=eps, min_samples=min_samples, metric="cosine").fit(embeddings)
        re_cluster_dict = {}
        for idx, label in enumerate(re_clustering.labels_):
            if label == -1:
                continue
            label_str = str(label)
            if label_str not in re_cluster_dict:
                re_cluster_dict[label_str] = []
            re_cluster_dict[label_str].append(sentences[idx])
        return [" | ".join(cluster) for cluster in re_cluster_dict.values()]

    def recursive_clustering(self, cluster_sentences, threshold, eps=0.22, min_samples=3, min_eps=0.02):
        current_eps = eps
        current_min_samples = min_samples
        new_clusters = [cluster_sentences]
        while True:
            next_clusters = []
            reclustered_any = False
            for cluster in new_clusters:
                if self.count_words(cluster) > threshold:
                    while current_eps >= min_eps:
                        reclustered = self.recluster_large_cluster(cluster, eps=current_eps, min_samples=current_min_samples)
                        if len(reclustered) > 1:
                            next_clusters.extend(reclustered)
                            reclustered_any = True
                            break
                        else:
                            if current_eps > min_eps:
                                current_eps -= 0.05
                    if len(reclustered) == 1:
                        next_clusters.append(cluster)
                else:
                    next_clusters.append(cluster)
            new_clusters = next_clusters
            if not reclustered_any:
                break
        return new_clusters

    def generate_predictions(self, dataset_exploded):
        tokenizer = self.tokenizer_my
        model = self.classification_model
        if self.device == torch.device("cuda"):
            model = model.half()

        reviews = dataset_exploded["sentences"]
        reviews = [str(review) for review in reviews if isinstance(review, str) and review.strip()]

        class ReviewDataset(TorchDataset):
            def __init__(self, reviews, tokenizer, max_len=128):
                self.reviews = reviews
                self.tokenizer = tokenizer
                self.max_len = max_len

            def __len__(self):
                return len(self.reviews)

            def __getitem__(self, idx):
                review = self.reviews[idx]
                encoding = self.tokenizer.encode_plus(
                    review,
                    add_special_tokens=True,
                    max_length=self.max_len,
                    return_token_type_ids=False,
                    padding='max_length',
                    truncation=True,
                    return_attention_mask=True,
                    return_tensors='pt'
                )
                return {key: val.flatten() for key, val in encoding.items()}

        dataset = ReviewDataset(reviews, tokenizer)
        batch_size = 32
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

        predictions = []

        from torch.cuda.amp import autocast

        for batch in tqdm(dataloader, desc="Предсказание отзывов"):
            batch = {key: val.to(self.device) for key, val in batch.items()}
            
            with torch.no_grad():
                with autocast():  # Используем смешанную точность
                    outputs = model(**batch)
                    logits = outputs[0] if isinstance(outputs, tuple) else outputs.logits
                    probabilities = torch.softmax(logits, dim=-1)
                    batch_predictions = (probabilities[:, 1] > 0.7).cpu().numpy()  # Используем порог 0.7
                    predictions.extend(batch_predictions)

        if len(predictions) != len(dataset_exploded):
            print(f"Warning: Length of predictions ({len(predictions)}) does not match length of index ({len(dataset_exploded)})")
            if len(predictions) < len(dataset_exploded):
                missing_count = len(dataset_exploded) - len(predictions)
                predictions.extend([0] * missing_count)
            elif len(predictions) > len(dataset_exploded):
                predictions = predictions[:len(dataset_exploded)]
        dataset_exploded = dataset_exploded.add_column("predictions", predictions)
        return dataset_exploded

    def process_group(self, category_name, product_name, group):
        all_sentences = group['sentences'].tolist()
        if not all_sentences:
            return pd.DataFrame()

        try:
            all_embeddings = self.compute_sentence_embeddings(all_sentences)
        except ValueError as e:
            print(f"Error in computing embeddings for product {product_name}: {e}")
            return pd.DataFrame()

        distance_matrix = squareform(pdist(all_embeddings, metric='cosine'))
        clustering = hdbscan.HDBSCAN(min_samples=3, metric='precomputed').fit(distance_matrix)

        cluster_dict = {}
        for idx, label in enumerate(clustering.labels_):
            if label == -1:
                continue
            label_str = str(label)
            if label_str not in cluster_dict:
                cluster_dict[label_str] = set()
            cluster_dict[label_str].add(all_sentences[idx])

        clusters = [" | ".join(sentences) for sentences in cluster_dict.values()]

        if not clusters:
            return pd.DataFrame()

        group['binary_rating'] = group['review_rating'].apply(lambda x: 1 if x in [4, 5] else 0)
        avg_rating = group['binary_rating'].mean()
        rating_category = 'positive' if avg_rating > 0.7 else 'neutral'
        rating_category = 'neutral' if avg_rating > 0.5 else 'negative'

        threshold = self.determine_threshold(clusters)

        final_clusters = []
        for cluster in clusters:
            if self.count_words(cluster) > threshold:
                final_clusters.extend(self.recursive_clustering(cluster, threshold))
            else:
                final_clusters.append(cluster)

        # Обеспечение минимального количества кластеров
        final_clusters = self.ensure_minimum_clusters(final_clusters, threshold)

        df_exploded_sorted = pd.DataFrame({
            'category': category_name,
            'product': product_name,
            'avg_rating': avg_rating,
            'rating_category': rating_category,
            'cluster_sentences': final_clusters
        })
        df_exploded_sorted['word_count'] = df_exploded_sorted['cluster_sentences'].apply(self.count_words)
        df_exploded_sorted['key_thought'] = df_exploded_sorted['cluster_sentences'].apply(self.extract_key_thought)
        df_exploded_sorted = df_exploded_sorted.sort_values(by='word_count', ascending=False)

        return df_exploded_sorted

    def determine_threshold(self, clusters):
        if len(clusters) == 1:
            cluster_word_count = self.count_words(clusters[0])
            if cluster_word_count > 20:
                return cluster_word_count / 2
            return cluster_word_count
        return np.min([np.mean([self.count_words(cluster) for cluster in clusters]) * 1.5, 250])

    def ensure_minimum_clusters(self, final_clusters, threshold):
        while len(final_clusters) < 3 and any(self.count_words(cluster) > threshold for cluster in final_clusters):
            largest_cluster = max(final_clusters, key=self.count_words)
            final_clusters.remove(largest_cluster)
            new_clusters = self.recursive_clustering(largest_cluster, threshold)
            if len(new_clusters) <= 1:
                final_clusters.append(largest_cluster)
                break
            final_clusters.extend(new_clusters)
        return final_clusters
    
    def cluster_reviews(self, dataset_exploded):
        # Фильтрация на основе предсказаний
        dataset_filtered = dataset_exploded.filter(lambda x: x['predictions'] == 1)
        
        # Преобразование в pandas DataFrame для группировки
        df_filtered = dataset_filtered.to_pandas()
        grouped = df_filtered.groupby(['category', 'product'])

        results = []
        
        # Последовательная обработка без параллелизма
        for (category_name, product_name), group in tqdm(grouped, desc="Processing categories and products"):
            result_df = self.process_group(category_name, product_name, group)
            if not result_df.empty:
                results.append(result_df)

        if results:  # Проверяем, что список results не пуст
            final_result = pd.concat(results, ignore_index=True)
            final_result = final_result[((final_result['word_count'] > 10) & (final_result['key_thought'].str.len() > 5))]
            final_result.to_csv("./reviews_keywords/feedbackfueltest.csv")
        else:
            print("No valid results to concatenate. Returning an empty DataFrame.")
            final_result = pd.DataFrame()  # Возвращаем пустой DataFrame, если нет данных для объединения
        
        return final_result

    def run(self):
        dataset_exploded = self.process_reviews()
        dataset_exploded = self.apply_embeddings(dataset_exploded)
        dataset_exploded = self.generate_predictions(dataset_exploded)
        result = self.cluster_reviews(dataset_exploded)
        return result


reviews_keywords = ReviewsKeywords(csv_path="./reviews_keywords/wildberries_reviews.csv",
                                    model_path='./reviews_keywords/fine_tuned_model')
final_result = reviews_keywords.run()
final_result.head()

ValueError: could not determine the shape of object type 'torch.storage.UntypedStorage'

In [ ]:
final_result

,category,product,avg_rating,rating_category,cluster_sentences,word_count,key_thought
0,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,0.819588,neutral,"Переднее колесо закрылось в снегу, подложили п...",40,"Переднее колесо закрылось в снегу, подложили п..."
1,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,0.819588,neutral,На вид крепкие. | Вроде прочные. | На вид проч...,12,На вид крепкие.
3,/Автотовары/OFFroad,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,0.842975,neutral,В деле не пробовал | В деле пока не пробовала....,37,В деле не пробовал
4,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,0.704545,neutral,"Рекомендую, буду брать еще | Закажу | мыМыочно...",20,Буду заказывать ещё.
5,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,0.704545,neutral,Еле пахнет. | Он даже не пахнет. | Пахнет каки...,13,Еле пахнет.
6,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,0.704545,neutral,🔥🔥🔥🔥🔥🔥 запах. | Запах огонь) | Запах огонь!!!!...,11,Запах 🔥!!!
7,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,0.853933,neutral,Убирает ржавчину хорошо через 10-20 минут | Со...,76,Ржавчину убирает отлично.
8,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,0.853933,neutral,"Ржавчина уже хорошо въелась, пришлось нескольк...",38,"Ржавчина уже хорошо въелась, пришлось нескольк..."
9,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,0.853933,neutral,"Фото «до» к сожалению не сделала, только «посл...",24,"Фото «до» к сожалению не сделала, только «после»"
10,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,0.853933,neutral,В деле пока не пробовала. | В деле не пробовал...,14,В деле не пробовал


In [ ]:
final_result

,category,product,avg_rating,rating_category,cluster_sentences,word_count,key_thought
0,/Спорт/Страйкбол и пейнтбол/Аксессуары,karbi / Рюкзак тактический туристический - кар...,0.797101,neutral,"Много доп карманов, чехол от дождя, прорезинен...",203,"Рюкзак вместительный, прочный, есть защитный ч..."
1,/Спорт/Страйкбол и пейнтбол/Аксессуары,karbi / Рюкзак тактический туристический - кар...,0.797101,neutral,"В подарок шёл компас,, налобныйфонарь,, ножане...",69,"В подарок положили фонарик налобный, компас и ..."


## Этап 1

In [ ]:
import cudf.pandas  # Импортирование cuDF и активация его использования
cudf.pandas.install()  # Установка cuDF как основного интерфейса для pandas
import os
import pandas as pd
from tqdm import tqdm
import torch
import pyarrow.parquet as pq
import dask.dataframe as dd

# # Чтение Parquet-файла с использованием pyarrow
# table = pq.read_table('./reviews_keywords/wildberries_reviews_corrected.parquet')

# # Преобразование в pandas DataFrame
# df_pandas = table.to_pandas()

# # Преобразование pandas DataFrame в Dask DataFrame
# df_dask = dd.from_pandas(df_pandas, npartitions=100)  # Укажите количество нужных вам частей
# df_pandas = None
# table = None
# import gc
# gc.collect()
# df_dask

In [ ]:
result = pd.read_csv("./reviews_keywords/wildberries_reviews.csv", nrows=1000)
result.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   Unnamed: 0        1000 non-null   int64
 1   review_full_text  1000 non-null   object
 2   review_rating     1000 non-null   int64
 3   product           1000 non-null   object
 4   category          1000 non-null   object
 5   url               1000 non-null   object
 6   corrected_text    1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 540.9+ KB


In [ ]:
# Оставляем только по 5 записей для каждого уникального значения в столбце 'product'
# result_limited = result.groupby('product').apply(lambda x: x.iloc[5:8]).reset_index(drop=True)
result_limited = result


In [ ]:
import spacy
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModel, BertTokenizerFast
import torch
from sklearn.cluster import DBSCAN
import numpy as np
from collections import Counter

# Проверка доступности GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, BertConfig
import hdbscan
from scipy.spatial.distance import pdist, squareform
from tqdm import tqdm

# Загрузка дообученной модели и токенизатора
# Загружаем конфигурацию модели


# Загрузка модели и токенайзера от Сбербанка
tokenizer = BertTokenizerFast.from_pretrained("./reviews_keywords/fine_tuned_model")
model = AutoModel.from_pretrained("./reviews_keywords/fine_tuned_model").to(device)
# Инициализируем модель с конфигурацией
config = BertConfig.from_pretrained('./reviews_keywords/fine_tuned_model', output_hidden_states=True)
model_classification = BertForSequenceClassification.from_pretrained('./reviews_keywords/fine_tuned_model', config=config).to(device)

        # self.tokenizer_my = BertTokenizerFast.from_pretrained(self.model_path)
        #  # Загрузка модели для классификации
        # self.classification_model = BertForSequenceClassification.from_pretrained(self.model_path).to(self.device)
        # # Загрузка базовой модели для получения эмбеддингов
        # self.embedding_model = AutoModel.from_pretrained(self.model_path).to(self.device)

spacy.prefer_gpu()
# Загрузка и настройка модели SpaCy
nlp = spacy.load("ru_core_news_lg", disable=["ner", "tagger", "attribute_ruler", "lemmatizer"])

df = result_limited

# Преобразование pandas DataFrame в Hugging Face Dataset
dataset = Dataset.from_pandas(df)

In [ ]:
import re

def clean_text(text):
    text = re.sub(r'[\n\r\t]+|\s{2,}', ' ', text)  # Объединяем шаги для замены пробелов
    text = re.sub(r'(?<!\.)\s*\.\s*|\s*\.\s*(?!\.)', '. ', text)  # Оптимизация замены точки
    return text.strip().rstrip('.')

def split_reviews_into_sentences(batch):
    # Очистка текстов
    cleaned_texts = [clean_text(text) for text in batch['corrected_text']]
    
    # Обработка текстов с помощью nlp.pipe с указанием batch_size
    docs = list(nlp.pipe(cleaned_texts, batch_size=64))  # Здесь 64 - пример значения

    # Извлечение предложений
    batch['sentences'] = [[sent.text for sent in doc.sents] for doc in docs]
    
    return batch

dataset = dataset.map(split_reviews_into_sentences, batched=True, batch_size=32)

# Преобразуем Dataset обратно в pandas DataFrame
df = dataset.to_pandas()

# Выполним explode по столбцу с предложениями
df_exploded = df.explode('sentences').reset_index(drop=True)

# Удаляем лишние столбцы, которые появились после explode
df_exploded = df_exploded.drop(columns=[col for col in df_exploded.columns if col.startswith('__index_level_')])

# Преобразуем DataFrame обратно в Hugging Face Dataset
dataset_exploded = Dataset.from_pandas(df_exploded)

from torch.cuda.amp import autocast

def compute_sentence_embeddings(sentences):
    # Фильтруем список, оставляя только строки
    sentences = [str(sentence) for sentence in sentences if isinstance(sentence, str)]
    
    if not sentences:
        raise ValueError("Input contains no valid strings.")

    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    
    with torch.no_grad():
        with autocast():  # Используем mixed precision для ускорения
            outputs = model_classification(**inputs)
    
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

def compute_embeddings_after_explode(batch):
    sentences = batch['sentences']

    # Проверяем, что все элементы в batch являются строками
    valid_sentences = [str(sentence) for sentence in sentences if isinstance(sentence, str)]
    
    if not valid_sentences:
        batch['sentence_embeddings'] = [[]] * len(sentences)  # Если нет валидных предложений, возвращаем пустые эмбеддинги
        return batch

    embeddings = compute_sentence_embeddings(valid_sentences)

    # Приведение эмбеддингов к типу float32 для консистентности
    embeddings = embeddings.astype(np.float32)

    # Проверяем, что количество эмбеддингов совпадает с количеством предложений
    if len(embeddings) != len(valid_sentences):
        raise ValueError("Количество эмбеддингов не совпадает с количеством предложений.")
    
    # Если количество предложений после фильтрации не совпадает с исходным, корректируем выходные данные
    final_embeddings = []
    embed_idx = 0
    for sentence in sentences:
        if isinstance(sentence, str):
            final_embeddings.append(embeddings[embed_idx])
            embed_idx += 1
        else:
            final_embeddings.append(np.zeros(embeddings.shape[1], dtype=np.float32))  # Добавляем нулевые эмбеддинги для невалидных предложений

    batch['sentence_embeddings'] = final_embeddings
    return batch

# Применение функции
dataset = dataset_exploded.map(compute_embeddings_after_explode, batched=True, batch_size=128)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2061 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizerFast, BertForSequenceClassification
from tqdm import tqdm
import os
import os
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, BertConfig
import nltk
from nltk.corpus import stopwords
import spacy
from tqdm import tqdm
import logging
import hdbscan  # HDBSCAN для более стабильной кластеризации с поддержкой кастомных метрик
from scipy.spatial.distance import pdist, squareform

In [ ]:
# Определение устройства (GPU или CPU)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Перевод модели в режим FP16, если это возможно
if use_cuda:
    model_classification = model_classification.half()

# Пример данных (замените на реальные данные)
reviews = dataset_exploded["sentences"]

# Очистка данных от некорректных значений
reviews = [str(review) for review in reviews if isinstance(review, str) and review.strip()]

# Создание кастомного Dataset для обработки отзывов
class ReviewDataset(Dataset):
    def __init__(self, reviews, tokenizer, max_len=128):
        self.reviews = reviews
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, idx):
        review = self.reviews[idx]
        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {key: val.flatten() for key, val in encoding.items()}

# Создаем датасет и DataLoader
dataset = ReviewDataset(reviews, tokenizer)
batch_size = 32  # Размер батча можно изменить в зависимости от объема доступной памяти GPU
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Получение предсказаний с отображением прогресса
predictions = []

from torch.cuda.amp import autocast  # Импортируем autocast для смешанной точности

for batch in tqdm(dataloader, desc="Предсказание отзывов"):
    batch = {key: val.to(device) for key, val in batch.items()}
    
    with torch.no_grad():
        with autocast():  # Используем смешанную точность
            outputs = model_classification(**batch)
            logits = outputs[0] if isinstance(outputs, tuple) else outputs.logits
            probabilities = torch.softmax(logits, dim=-1)
            batch_predictions = (probabilities[:, 1] > 0.7).cpu().numpy()  # Используем порог 0.7
            predictions.extend(batch_predictions)

# Преобразование в DataFrame, если это еще не сделано
if not isinstance(dataset_exploded, pd.DataFrame):
    dataset_exploded = pd.DataFrame(dataset_exploded)

# Проверка и обработка несоответствия длины
if len(predictions) != len(dataset_exploded):
    print(f"Warning: Length of predictions ({len(predictions)}) does not match length of index ({len(dataset_exploded)})")
    
    # Пример: Заполнение недостающих значений
    if len(predictions) < len(dataset_exploded):
        missing_count = len(dataset_exploded) - len(predictions)
        predictions.extend([0] * missing_count)  # Добавляем нули в случае недостатка предсказаний

    elif len(predictions) > len(dataset_exploded):
        predictions = predictions[:len(dataset_exploded)]  # Обрезаем список предсказаний

# Присоединение предсказаний к датасету
dataset_exploded['predictions'] = predictions
dataset_exploded.head()



Предсказание отзывов:   0%|                                                                                                                                                                        | 0/65 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid d

,Unnamed: 0,review_full_text,review_rating,product,category,url,corrected_text,sentences,__index_level_0__,predictions
0,0,Работает хорошо.,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Работает хорошо.,Работает хорошо,0,False
1,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...","Пришло быстро, все целое на вид.",1,False
2,1,"Пришло быстро, все целое на вид. Завтра буду и...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Пришло быстро, все целое на вид. Завтра буду и...",Завтра буду испытывать,2,True
3,2,"Купил на квадр для поднятия отвала, установка ...",5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,"Купил на квадр для поднятия отвала, установка ...","Купил на квадр для поднятия отвала, установка ...",3,True
4,3,Лебёдка хорошая. Но в инструкции ни слова про ...,5,Shtapler / Лебедка электрическая 12v 3000lb 13...,/Автотовары/OFFroad,https://www.wildberries.ru/catalog/162315454/f...,Лебёдка хорошая. Но в инструкции ни слова про ...,Лебёдка хорошая.,4,True


In [ ]:
# Настройка логирования
logging.basicConfig(filename='./reviews_keywords/clustering.log', 
                    level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Загрузка модели spaCy для русского языка
nlp = spacy.load("ru_core_news_lg")

# Установка стоп-слов
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
total_words_before = 0
total_words_thrown = 0
total_words_after = 0

# Перевод модели в режим FP16, если это возможно
if torch.cuda.is_available():
    model_classification = model_classification.half()

from sklearn.neighbors import NearestNeighbors

def automatic_dbscan_params(embeddings, target_percentile=0.8, min_samples_factor=0.1):
    n_samples = len(embeddings)
    logging.info(f"n_samples: {n_samples}")
    n_neighbors = min(15, n_samples)
    
    neighbors = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    neighbors_fit = neighbors.fit(embeddings)
    distances, indices = neighbors_fit.kneighbors(embeddings)
    
    eps = np.percentile(distances[:, n_neighbors - 1], target_percentile)
    
    # Убедимся, что eps положительное значение
    if eps <= 0:
        eps = 0.001  # Задаем минимально допустимое значение
    
    min_samples = 4 #max(int(n_samples * min_samples_factor), 2)
    
    return eps, min_samples


def find_centroid(embeddings):
    return np.mean(embeddings, axis=0)

def compute_sentence_embeddings(sentences):
    if not all(isinstance(sentence, str) and sentence.strip() for sentence in sentences):
        raise ValueError("All items in the input must be non-empty strings.")
    
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model_classification(**inputs)
        if outputs.hidden_states is None:
            raise ValueError("Модель не возвращает скрытые состояния. Проверьте конфигурацию модели.")
        hidden_states = outputs.hidden_states[-1]
    embeddings = hidden_states.mean(dim=1).cpu().numpy()
    return embeddings

def extract_key_thought(cluster_sentences):
    sentences = cluster_sentences.split(" | ")
    embeddings = compute_sentence_embeddings(sentences)
    centroid = find_centroid(embeddings)
    similarities = cosine_similarity(embeddings, [centroid])
    key_sentence_index = np.argmax(similarities)
    return sentences[key_sentence_index]

def count_words(cluster_sentences):
    words = cluster_sentences.split()
    return len(words)

def recluster_large_cluster(cluster_sentences, eps=0.1, min_samples=2):
    sentences = cluster_sentences.split(" | ")
    embeddings = compute_sentence_embeddings(sentences)
    
    re_clustering = DBSCAN(eps=eps, min_samples=min_samples, metric="cosine").fit(embeddings)
    
    re_cluster_dict = {}
    local_words_thrown = 0
    
    for idx, label in enumerate(re_clustering.labels_):
        if label == -1:
            local_words_thrown += count_words(sentences[idx])  # Учет слов, отнесенных к шуму
            continue
        label_str = str(label)
        if label_str not in re_cluster_dict:
            re_cluster_dict[label_str] = []
        re_cluster_dict[label_str].append(sentences[idx])
    
    return [" | ".join(cluster) for cluster in re_cluster_dict.values()], local_words_thrown

def dynamic_recursive_clustering(cluster_sentences, threshold, max_iterations=10):
    new_clusters = [cluster_sentences]
    iteration = 0
    total_words_thrown_recursive = 0

    while iteration < max_iterations:
        next_clusters = []
        reclustered_any = False
        
        for cluster in new_clusters:
            if count_words(cluster) > threshold:
                sentences = cluster.split(" | ")
                embeddings = compute_sentence_embeddings(sentences)
                current_eps, current_min_samples = automatic_dbscan_params(embeddings)

                reclustered, words_thrown = recluster_large_cluster(cluster, eps=current_eps, min_samples=current_min_samples)
                total_words_thrown_recursive += words_thrown
                
                if len(reclustered) > 1:
                    next_clusters.extend(reclustered)
                    reclustered_any = True
                else:
                    next_clusters.append(cluster)
                logging.info(f"current_eps: {current_eps} current_min_samples: {current_min_samples} count_words(cluster): {count_words(cluster)}")
            else:
                next_clusters.append(cluster)
        
        new_clusters = next_clusters
        iteration += 1

        if not reclustered_any:
            break

    return new_clusters, total_words_thrown_recursive

# Основной процесс кластеризации по категориям и продуктам
def count_words_before_clustering(sentences):
    words = " ".join(sentences).split()
    return len(words)

final_result = pd.DataFrame()

for (category_name, product_name), group in tqdm(dataset_exploded[dataset_exploded["predictions"] == 1].groupby(['category', 'product']), desc="Processing categories and products"):
    all_sentences = group['sentences'].tolist()

    if not all_sentences:
        continue

    # Подсчет слов до кластеризации
    total_words_before += count_words_before_clustering(all_sentences)

    all_embeddings = compute_sentence_embeddings(all_sentences)
    
    eps, min_samples = automatic_dbscan_params(all_embeddings)

    clustering = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine').fit(all_embeddings)
    
    cluster_dict = {}
    local_words_thrown = 0
    
    for idx, label in enumerate(clustering.labels_):
        if label == -1:
            local_words_thrown += count_words(all_sentences[idx])  # Учет слов, отнесенных к шуму
            continue
        label_str = str(label)
        if label_str not in cluster_dict:
            cluster_dict[label_str] = set()
        cluster_dict[label_str].add(all_sentences[idx])

    total_words_thrown += local_words_thrown

    clusters = [" | ".join(sentences) for sentences in cluster_dict.values()]

    if not clusters:
        continue

    group['binary_rating'] = group['review_rating'].apply(lambda x: 1 if x in [4, 5] else 0)
    avg_rating = group['binary_rating'].mean()
    rating_category = 'positive' if avg_rating > 0.7 else 'neutral' if avg_rating > 0.5 else 'negative'

    threshold = np.min([np.mean([count_words(cluster) for cluster in clusters]) * 1.5, 250])

    final_clusters = []
    for cluster in clusters:
        if count_words(cluster) > threshold:
            reclustered_clusters, words_thrown_recursive = dynamic_recursive_clustering(cluster, threshold)
            total_words_thrown += words_thrown_recursive
            final_clusters.extend(reclustered_clusters)
        else:
            final_clusters.append(cluster)

    df_exploded_sorted = pd.DataFrame({
        'category': category_name,
        'product': product_name,
        'avg_rating': avg_rating,
        'rating_category': rating_category,
        'cluster_sentences': final_clusters
    })
    
    df_exploded_sorted['word_count'] = df_exploded_sorted['cluster_sentences'].apply(count_words)
    total_words_after += df_exploded_sorted['word_count'].sum()
    
    df_exploded_sorted['key_thought'] = df_exploded_sorted['cluster_sentences'].apply(extract_key_thought)
    df_exploded_sorted = df_exploded_sorted.sort_values(by='word_count', ascending=False)

    final_result = pd.concat([final_result, df_exploded_sorted], ignore_index=True)

# Показать результат
display(f"Total words before clustering: {total_words_before}")
display(f"Total words thrown away during clustering: {total_words_thrown}")
display(f"Total words after clustering: {total_words_after}")
display(final_result[['category', 'product', 'avg_rating', 'rating_category', 'cluster_sentences', 'key_thought', 'word_count']])


Processing categories and products: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:04<00:00,  3.65it/s]


'Total words before clustering: 8981'

'Total words thrown away during clustering: 5445'

'Total words after clustering: 3894'

,category,product,avg_rating,rating_category,cluster_sentences,key_thought,word_count
0,/Автотовары/OFFroad,Hangkai / Лебедка электрическая влагозащитная ...,0.873016,positive,"Провод плюсовой тонкий 10мм, лучше сразу замен...","Рычаг свободного хода лебедки не герметичный, ...",335
1,/Автотовары/OFFroad,MOTORin / Расширитель колёсных арок 40 мм,0.937500,positive,без каркасной проволоки и пластинки ставятся л...,"Для установки требуется частое сверление арок,...",170
2,/Автотовары/OFFroad,MOTORin / расширители арок,0.722222,positive,Расширители хорошие вот только пришлось для фо...,На вид огонь 🔥ещё не ставил,138
3,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 3000lb 13...,0.647059,neutral,"Купил на квадр для поднятия отвала, установка ...","Отличная лебедка, но не подошла",138
4,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 3500lb 15...,0.843750,positive,"Хорошая лебедка, со своей задачей справляется ...","Нормальный лебёдка, но шумел, разобрался полож...",233
5,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 4500lb 20...,0.843750,positive,"Хорошая лебедка, со своей задачей справляется ...","Нормальный лебёдка, но шумел, разобрался полож...",233
6,/Автотовары/OFFroad,Vixem / Лебедка автомобильная 12 v 4000 1814 к...,1.000000,positive,"Один метр троса зажали, смазки не много и та г...","Один метр троса зажали, смазки не много и та г...",41
7,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,0.819588,positive,"Не думаю, что газель выдержит у меня газель-4 ...",Зимняя резина пережевывает эти пластиковые ант...,431
8,/Автотовары/OFFroad,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,0.842975,positive,Иногда их выбрасывало из-под колес. | Хватало ...,"Не выручать даже летом, чуток сел в небольшую ...",371
9,/Автотовары/OFFroad,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,0.842975,positive,"В деле не пробовал | Траки мощные, в деле ещё ...",В деле пока не пробовала.,37


In [ ]:
final_result.sort_values("word_count", ascending=False)

,category,product,avg_rating,rating_category,cluster_sentences,word_count,key_thought
8,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 3500lb 15...,0.843750,positive,"Хорошая лебедка, со своей задачей справляется ...",138,"Нормальный лебёдка, но шумел, разобрался полож..."
9,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 4500lb 20...,0.843750,positive,"Хорошая лебедка, со своей задачей справляется ...",138,"Нормальный лебёдка, но шумел, разобрался полож..."
2,/Автотовары/OFFroad,MOTORin / Расширитель колёсных арок 40 мм,0.937500,positive,Пришло с опозданием на два дня позже на ниву в...,83,"Сами расширители арок нормальные, но ставить н..."
6,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 3000lb 13...,0.647059,neutral,Но в инструкции ни слова про сборку и креплени...,60,"При первой поездки в лес, барабан отломился ко..."
3,/Автотовары/OFFroad,MOTORin / расширители арок,0.722222,positive,Расширители хорошие вот только пришлось для фо...,47,На вид огонь 🔥ещё не ставил
10,/Автотовары/OFFroad,Vixem / Лебедка автомобильная 12 v 4000 1814 к...,1.000000,positive,"Один метр троса зажали, смазки не много и та г...",41,"Один метр троса зажали, смазки не много и та г..."
11,/Автотовары/OFFroad,Фрегат Лифт Подвеска / Лифт комплект рессоры К...,0.750000,positive,"На вид не плохие, но к сожалению на уаз профи ...",41,Встали как родные устанавливал на петро пикапа
4,/Автотовары/OFFroad,MOTORin / расширители арок,0.722222,positive,"Монтажный комплект не полный, должна быть ещё ...",22,"Монтажный комплект не полный, должна быть ещё ..."
0,/Автотовары/OFFroad,Autobrand_AED / Дополнительная led фара 30w с ...,0.333333,negative,Светят хорошо | Пользоваться невозможно | Но ч...,19,"Но через месяц сгорел один диод, второй ещё че..."
1,/Автотовары/OFFroad,MOTORin / Расширители арок 60 Эк,1.000000,positive,Отличная резинка на откатные ворота!!! | Должн...,12,"Должно хватить длины, пока не установлено!"


In [ ]:
# Удаление записей с word_count <= 10 и ключевой мыслью менее 3 символов
final_result = final_result[((final_result['word_count'] > 10) & (final_result['key_thought'].str.len() > 5))]
final_result

,category,product,avg_rating,rating_category,cluster_sentences,word_count,key_thought
0,/Автотовары/OFFroad,ВПМ / Антибукс - антипробуксовочные траки утол...,0.819588,positive,"Не вставать сзади, когда машина начинает движе...",1738,"2 раза спасали на гололедице, шлифовал на мест..."
2,/Автотовары/OFFroad,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,0.842975,positive,Иногда их выбрасывало из-под колес. | Хватало ...,442,"Не выручать даже летом, чуток сел в небольшую ..."
3,/Автотовары/OFFroad,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,0.842975,positive,"В деле не пробовал | Траки мощные, в деле ещё ...",37,В деле пока не пробовала.
4,/Автотовары/Автокосметика и автохимия,Пахнет и Точка / Ароматизатор в машину автопар...,0.704545,positive,Ужасная вонючка. | Я обычно не пишу плохие отз...,1120,"Во-первых пришел совсем другой аромат, во втор..."
6,/Автотовары/Автокосметика и автохимия,ПолиКомПласт / Преобразователь очиститель ржав...,0.853933,positive,Ржавчина леш добавилась. | минус 5 звёзд | Луч...,1370,"На унитазе ржавчину убрало на раз два, чем до ..."


In [ ]:
final_result.to_csv("./reviews_keywords/feedbackfueltest.csv")